In [ ]:
#INTEL CORPORATION CONFIDENTIAL AND PROPRIETARY
#
# Copyright © 2020-2021 Intel Corporation.
# 
# This software and the related documents are Intel copyrighted
# materials, and your use of them is governed by the express 
# license under which they were provided to you (License). Unless
# the License provides otherwise, you may not use, modify, copy, 
# publish, distribute, disclose or transmit  this software or the
# related documents without Intel's prior written permission.
# 
# This software and the related documents are provided as is, with
# no express or implied warranties, other than those that are 
# expressly stated in the License.

# Import modules

In [2]:
import os
import sys
sys.path.append(os.path.expanduser("~") + "/nxsdk-nxsdk")

import time
import numpy as np
import nxsdk.api.n2a as nx
import nxsdk_modules.slayer.src as nxSlayer
from gestureDataset import Event

os.environ['SLURM'] = '1' 
os.environ['PARTITION'] = 'nahuku32'

# Define dataset

This is defined here just for demo. Actually __you are supposed to import the dataset module that you defined during SLAYER training__.

In [3]:
class IBMGestureDataset():
    """Interface to IBM Gesture Test Dataset
    """
    def __init__(self, path):
        self.path = path

        testParams = np.loadtxt(self.path + '/test.txt').astype('int')
        self.fileNumbers = testParams[:, 0]
        self.labels = testParams[:, 1]
            
    def __getitem__(self, index):
        filename = self.path + '/' + str(self.fileNumbers[index]) + '.bs2'
        data = Event(filename)
        label = self.labels[index]
        npEvent = np.zeros((len(data.x), 4))
        npEvent[:,0] = data.x
        npEvent[:,1] = data.y
        npEvent[:,2] = data.p
        npEvent[:,3] = data.t

        return npEvent, label

    def __len__(self):
        return len(self.labels)



# Create nxSlayer Dataset instance

In [4]:
dataset = nxSlayer.auto.s2lDataset(IBMGestureDataset(path='/nfs/ncl/datasets/DVSgesture'))

# Create network

In [5]:
# net = nxSlayer.auto.Network('model.net')
net = nxSlayer.auto.Network('model_delay.net')

# Compile, setup input-output, and run the network

In [6]:
boardName = 'dvs_gesture'
loadSavedBoard = False # True # load precompiledBoard or compile again

net.create()

if loadSavedBoard is True:
    net.compile(boardName)
    net.setupIO(dataset, numSnips=3)
else:
    net.compile()
    net.setupIO(dataset, numSnips=3)
    net.save(boardName) # this will save the compiled board
    
results = net.run()

Creating Network
|   Type   |  W  |  H  |  C  | ker | str | pad |delay|neurons/Core|cores|
|input     |  128|  128|    2|     |     |     |False|        1024|   32|
|pool      |   32|   32|    2|    4|     |     |False|         256|    8|
|conv      |   32|   32|   16|    5|    1|    2|True |         256|   64|
|pool      |   16|   16|   16|    2|     |     |False|         341|   13|
|conv      |   16|   16|   32|    3|    1|    1|True |         256|   32|
|pool      |    8|    8|   32|    2|     |     |False|         340|    7|
|dense     |    1|    1|  512|     |     |     |True |          46|   12|
|dense     |    1|    1|   11|     |     |     |False|         500|    1|
|Total                                                            |  169|

Starting compilation ...
Completed compilation in 284.03 seconds
Using per sample spike time: 1500steps (+ 100steps gap)
INFO:DRV:  SLURM is being run in background
INFO:DRV:  Connecting to 134.134.68.92:34863
INFO:DRV:      Host server up...

# Check accuracy

In [7]:
labels = dataset.labels
accuracy = nxSlayer.s2l.checkAccuracy(labels, results)
print('Final accuracy is {:.2f}%'.format(accuracy*100))

Final accuracy is 97.73%
